<a href="https://colab.research.google.com/github/KarbivnychyiVolodymyr/machine_learning/blob/main/%D0%9A%D0%B0%D1%80%D0%B1%D1%96%D0%B2%D0%BD%D0%B8%D1%87%D0%B8%D0%B9_%D0%BF%D1%80_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357266 sha256=b7e162f580cfc87f45cc1cd53762d12457d158166c045a717cff7faad95dfb97
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
'''Завдання 1.
1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки
Surprise.'''
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

data = Dataset.load_builtin("ml-100k")

trainset, testset = train_test_split(data, test_size=0.25)
algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions)


Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
RMSE: 0.9348


0.934814058754397

In [3]:
'''2. Виведіть перші 5 рядків завантаженого датасету.'''
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')

print("Перші 5 рядків датасету:")
for row in data.raw_ratings[:5]:
    user_id, movie_id, rating, timestamp = row
    print(f"Користувач {user_id} оцінив фільм {movie_id} на {rating} балів (часова позначка: {timestamp})")

Перші 5 рядків датасету:
Користувач 196 оцінив фільм 242 на 3.0 балів (часова позначка: 881250949)
Користувач 186 оцінив фільм 302 на 3.0 балів (часова позначка: 891717742)
Користувач 22 оцінив фільм 377 на 1.0 балів (часова позначка: 878887116)
Користувач 244 оцінив фільм 51 на 2.0 балів (часова позначка: 880606923)
Користувач 166 оцінив фільм 346 на 1.0 балів (часова позначка: 886397596)


In [4]:
'''3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього
датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.'''
'''4. Використайте крос-валідацію для підбору оптимальних параметрів для
обох алгоритмів.'''
trainset, testset = train_test_split(data, test_size=0.2)

# Параметри для підбору через GridSearchCV для SVD
param_grid_svd = {
    'n_factors': [25, 100],
    'lr_all': [0.005, 0.015],
    'reg_all': [0.02, 0.05]
}

# Параметри для підбору через GridSearchCV для KNNBasic
param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 3],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=3)
grid_search_knn.fit(data)



Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Comput

In [5]:
'''5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки
(MAE).'''

best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

# Вибір моделі з найменшим MAE
if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")


Найкращий MAE для SVD: 0.7273358441539387
Найкращий MAE для KNNBasic: 0.8081718538423711
Вибраний алгоритм: SVD


In [6]:
'''6. Виведи рекомендації (10 фільмів) для конкретного користувача.'''

best_model.fit(trainset)

# Виведення рекомендацій для конкретного користувача
user_id = str(174)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

# Отримання рекомендацій
all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих фільмів
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 174: 145
Топ-10 фільмів, рекомендованих для користувача:
Фільм 923 з прогнозованим рейтингом 5.00
Фільм 170 з прогнозованим рейтингом 5.00
Фільм 750 з прогнозованим рейтингом 5.00
Фільм 1367 з прогнозованим рейтингом 5.00
Фільм 1449 з прогнозованим рейтингом 5.00
Фільм 306 з прогнозованим рейтингом 5.00
Фільм 169 з прогнозованим рейтингом 4.96
Фільм 316 з прогнозованим рейтингом 4.91
Фільм 275 з прогнозованим рейтингом 4.90
Фільм 127 з прогнозованим рейтингом 4.89


In [7]:
'''Завдання 2.
1. Побудуйте власну рекомендаційну систему (приклад є в документації).
Оцініть її.'''
from surprise import NMF, NormalPredictor
from surprise.model_selection import cross_validate

# Алгоритм 1: NMF (Non-negative Matrix Factorization)
algo_nmf = NMF()

# Алгоритм 2: Random (випадковий прогноз)
algo_random = NormalPredictor()

# Крос-валідація для NMF
cv_nmf = cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Крос-валідація для випадкового прогнозу
cv_random = cross_validate(algo_random, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

print(f"Результати для NMF:\nRMSE: {cv_nmf['test_rmse'].mean()}\nMAE: {cv_nmf['test_mae'].mean()}")
print(f"Результати для випадкового прогнозу:\nRMSE: {cv_random['test_rmse'].mean()}\nMAE: {cv_random['test_mae'].mean()}")


Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9634  0.9631  0.9663  0.9603  0.9636  0.9634  0.0019  
MAE (testset)     0.7576  0.7593  0.7573  0.7543  0.7564  0.7570  0.0016  
Fit time          2.57    2.16    2.46    3.10    2.48    2.55    0.31    
Test time         0.31    0.10    0.27    0.10    0.25    0.21    0.09    
Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5201  1.5287  1.5239  1.5073  1.5334  1.5227  0.0089  
MAE (testset)     1.2197  1.2332  1.2245  1.2088  1.2317  1.2236  0.0088  
Fit time          0.11    0.16    0.18    0.18    0.18    0.16    0.03    
Test time         0.09    0.25    0.10    0.26    0.12    0.16    0.08    
Результати для NMF:
RMSE: 0.9633509298294272
MAE: 0.7569707972603362
Результати для випадкового прогнозу:
RMSE: 1.5226780203382897

In [8]:
'''2. Отримайте рекомендацію для певного користувача.'''
# Тренуємо модель NMF на тренувальному наборі
algo_nmf.fit(trainset)

user_id = str(174)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]

# Прогнозуємо для фільмів, які користувач ще не бачив
unrated_items = all_items - rated_items
predictions = [(item, algo_nmf.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих рекомендацій
print(f"Топ-10 рекомендацій для користувача {user_id}:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")


Топ-10 рекомендацій для користувача 174:
Фільм 511 з прогнозованим рейтингом 5.00
Фільм 174 з прогнозованим рейтингом 5.00
Фільм 478 з прогнозованим рейтингом 5.00
Фільм 156 з прогнозованим рейтингом 5.00
Фільм 127 з прогнозованим рейтингом 5.00
Фільм 654 з прогнозованим рейтингом 5.00
Фільм 856 з прогнозованим рейтингом 5.00
Фільм 169 з прогнозованим рейтингом 5.00
Фільм 1639 з прогнозованим рейтингом 5.00
Фільм 489 з прогнозованим рейтингом 5.00


In [ ]:
'''Висновки:

У ході виконання лабораторної роботи були розглянуті принципи функціонування
рекомендаційних систем на основі бібліотеки Surprise. Ми використали датасет
рецензій (ml-100k) та реалізували два алгоритми: SVD і KNNBasic. Для обох
алгоритмів було виконано крос-валідацію для вибору оптимальних параметрів, що
дозволило покращити точність прогнозів. За метрикою MAE було обрано
найефективніший алгоритм, який було використано для генерації рекомендацій
для окремого користувача.

У другій частині роботи ми створили власну рекомендаційну систему, застосувавши
алгоритм NMF (Non-negative Matrix Factorization), і провели її оцінку.
Порівняння результатів NMF із випадковим прогнозуванням продемонструвало
перевагу методів факторизації матриць для розробки ефективних рекомендаційних
систем. Загалом, ця робота дозволила ознайомитися з ключовими підходами до
створення рекомендаційних систем, а також з методами налаштування та оцінки
моделей за різними метриками.'''